# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
from __future__ import print_function
import os
import flopy

Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("..", "data", "freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False)


We can see the ``SpatialReference`` instance has generic entries, as does ``start_datetime``

In [3]:
ml.dis.sr

xul:0; yul:10000; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2; length_multiplier:1.0

In [4]:
ml.dis.start_datetime

'1/1/1970'

Setting the attributes of the ``ml.dis.sr`` is easy:

In [5]:
ml.dis.sr.xul = 123456.7
ml.dis.sr.yul = 765432.1
rotation = 15.0
proj4_str = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
ml.dis.start_datetime = '7/4/1776'
ml.dis.sr

xul:123456.7; yul:765432.1; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2; length_multiplier:1.0

In [6]:
ml.dis.start_datetime

'7/4/1776'

### Some netCDF export capabilities:

#### Export the whole model (inputs and outputs)

In [7]:
# make directory
pth = os.path.join('data', 'netCDF_export')
if not os.path.exists(pth):
    os.makedirs(pth)

In [8]:
fnc = ml.export(os.path.join(pth, ml.name+'.in.nc'))
flopy.export.utils.output_helper(os.path.join(pth, ml.name+'.out.nc'), ml, ml.load_results(as_dict=True))

error reading output filenames from OC package:list index out of range


AssertionError: 

export a single array to netcdf or shapefile

In [ ]:
# export a 2d array
ml.dis.top.export(os.path.join(model_ws, 'top.nc'))
ml.dis.top.export(os.path.join(model_ws, 'top.shp'))

#### Export a 3d array

In [ ]:
#export a 3d array
ml.upw.hk.export(os.path.join(model_ws, 'hk.nc'))
ml.upw.hk.export(os.path.join(model_ws, 'hk.shp'))

#### Export a number of things to the same netCDF file

In [ ]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(model_ws, 'test.nc'))
ml.upw.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


### Export whole packages to a netCDF file

In [ ]:
# export mflist
fnc = ml.wel.export(os.path.join(model_ws, 'packages.nc'))
ml.upw.export(fnc)
fnc.nc

### Export the whole model to a netCDF

In [ ]:
fnc = ml.export(os.path.join(model_ws, 'model.nc'))
fnc.nc